# Pure API Demonstration: Persons

These notebooks demonstrate some uses of the API of Elsevier's *Pure* Current Research Information System (CRIS). This notebook demonstrates some requests for research software.

Research Software is currently recorded in Pure as a type of Research Output.

**Enter API details - including an API key which gives access to the `persons` and `organisational-units` endpoints - in [`_Config_DO_THIS_FIRST.ipynb`](./_Config_DO_THIS_FIRST.ipynb) and execute that notebook before executing this notebook.**

In [ ]:
# We're using the requests library to talk to the API
import requests

# We need the json library to manipulate json
import json

# We'll use the bokeh library to generate our visualisations (we need math to rotate labels 90 degrees)
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import PuBu
import math

# The display, HTML and Markdown libraries will help render HTML and Markdown
from IPython.core.display import display, HTML, Markdown

# The utility_functions.py script includes:
# - pretty_print_json(json_object, ind=4) - prints json with indentation and colours
import utility_functions as uf

In [ ]:
# Retrieve the api_url and headers set in the config notebook
%store -r api_url
%store -r headers

In [ ]:
# We'll be making requests to /organisational-units and /persons
orgs_request_url = "/".join([api_url,"organisational-units"])
persons_request_url = "/".join([api_url,"persons"])

# We'll be making POST requests, so we need to modify the default headers
# to indicate how we're formatting the data we're sending
post_headers = headers.copy()
post_headers["Content-Type"] = 'application/json'

## Proportion of people in each School with an ORCID

### Identify the schools

First, we need to use the `organisational-units` endpoint to get a list of schools.

In [ ]:
# Create the JSON structure using dicts/lists
# We only need the identifiers and names
# We only want current organisations
# Ordering alphabetically by name
org_request_body = {
    "organisationalUnitTypeUris": [
        "/dk/atira/pure/organisation/organisationtypes/organisation/school"
    ],
    "orderings": [
        "name"
    ],
    "fields": [
        "uuid",
        "name.value"
    ],
    "organisationalUnitPeriodStatus": "ACTIVE",
    "size": 50
}
# Serialize as JSON
org_request_json = json.dumps(org_request_body)

# Make the request
org_response = requests.post(url=orgs_request_url, headers=post_headers, data=org_request_json)
schools_json = org_response.json()

# Display details
for item in schools_json["items"]:
    print(item["uuid"],item["name"][0]["value"])

### Get proportion of people with ORCIDs by School

In [ ]:
# Create the JSON structure using dicts/lists
# We only need uuids and orcids
# We only want current people
persons_request_body = {
  "employmentStatus": "ACTIVE",
  "fields": [ "uuid", "orcid"],
  "forOrganisations": {
    "uuids": [""],
    "organisationalUnitPeriodStatus": "ACTIVE"
  },
  "size": 2000
}

orcid_pcs = {}

for school in schools_json["items"]:
    
    # Specify the school in the request
    persons_request_body["forOrganisations"]["uuids"][0] = school["uuid"]
    
    #Serialise the request as JSON
    persons_request_json = json.dumps(persons_request_body)
    
    # Make the request
    persons_response = requests.post(url=persons_request_url, headers=post_headers, data=persons_request_json)
    persons_json = persons_response.json()
        
    # Count the total number of people and those with ORCIDs
    total_people = persons_json["count"]
    
    people_with_orcids = 0
    for person in persons_json["items"]:
        if "orcid" in person:
            people_with_orcids += 1
        
    orcid_pcs[school["name"][0]["value"]] = people_with_orcids / total_people * 100
    
print(orcid_pcs)

### Visualise the results

In [ ]:
output_notebook()

# x labels need to be strings
x_labels = list(map(str, list(orcid_pcs.keys())))
y_values = list(orcid_pcs.values())

p = figure(x_range=x_labels, plot_height=500, title="% of people with ORCIDs by school")
p.vbar(x=x_labels, top=y_values, width=0.9, color=PuBu[7][2])

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 100
p.xaxis.major_label_orientation = math.pi/2

show(p)